# Projeto

**Objetivo**: Desenvolver um CRUD utilizando um arquivo JSON para "persistir" os dados e aplicar algumas outras funcionalidades que seriam interessantes no sistema.

**Requisitos**: O projeto deve conter as seguintes funcionalidades/ características:

- Utilizar JSON para persistir os dados
- Função de criar o arquivo JSON (amanda) OK
- Função para adicionar registros (através do *input*, ou de um conjunto de dados históricos, por exemplo) (ricardo)
    - Não deixar para o usuário decidir o id, apenas coloque o proximo valor disponível
- Função para realizar o *udpate* (atualização) das informações (leticia)
    - Selecionar pelo ID
- Função para excluir registros (adriely)
    - Selecionar pelo ID
- Função para realizar a leitura de registros (printar os dados na tela do consumidor x, lista) (leticia)
- Função para filtrar informações dos dados (Amanda) +/- OK
- Função para aplicar alguma regra de negócio (varia de acordo com o sistema, exemplo, grupo criou um sistema de consulta de colaboradores, logo deseja-se ter uma função que calcula o faturamento de todos os funcionários que são vendedores) (lorrany)
    - Utilizar as funções de filtro e de estatística
- Função que retorne alguma estatística dos dados (lorrany)
- menu (daniel)
- Exemplos de funcionamento de cada uma das funções acima

**Pontos importantes**:

- Organização de código no ``jupyter notebook`` (não precisa de ppt, no dia apenas o notebook será necessário)
- Definição sobre o que será o sistema (livre e definido pelo grupo)
- Apresentação do sistema e funcionalidades (assim como os exemplos). Não precisa ser o grupo todo apresentando
- Gerenciamento de erros, como no caso de num input o usuário colocar 'abc' num campo de valor inteiro
- A nota do grupo será a nota de cada integrante
- Tempo de apresentação: de 15 a 20 minutos
- A princípio todos integrantes devem entregar o mesmo notebook no sistema do LMS, para fins de retorno de nota
- Não precisa se ater somente aos 10 requisitos acima, pode ir além :)

banco de dados: 
https://www.kaggle.com/datasets/iamsouravbanerjee/customer-shopping-trends-dataset

Pacotes necessários:
(**favor instalar!**)
- ``os``
- ``translate``
- ``json``
- ``csv``
- ``langdetect``

## Função para validação de respostas do usuário

In [34]:
def validacao_respostas(pergunta, respostas_validas):
  """
  Validação de respostas. A função valida respostas de um caracter
  É passado como parâmetro:
    - A pergunta que se quer fazer (str)
    - Uma lista com as respostas válidas

  Caso o usuário digite algo diferente das respostas válidas,
  a função repetirá a pergunta até que o usuário responda um valor válido.
  E só retornará opções válidas de valores
  """
  print(pergunta)
  resp = input().upper()[0]# apenas 1ªletra
  pergunta = 'Resposta inválida! ' + pergunta
  while not resp in respostas_validas:
    print(pergunta)
    resp = input().upper()[0]# apenas 1ªletra
    #resp = input(pergunta).upper()[0]# apenas 1ªletra
  return resp

## Função que lista os arquivos disponíveis

In [35]:
import os
def arquivos_disponiveis(extensao: str):
    """Função recebe uma extensão de arquivo e lista todos os arquivos com a extensão que estão no diretório

    Args:
        extensao (str): extensão do arquivo

    Returns:
        opcoes (dict): as opções de arquivos disponíveis
    """
    #pergunta = 'Informe o numero do arquivo que deseja importar'
    opcoes = dict()
    arquivos = os.listdir() # lista todos os arquivos presentes na pasta
    arquivos_extensao = [arquivo for arquivo in arquivos if arquivo.endswith(extensao)] # filtra pela extensao
    print(f'Lista de arquivos {extensao.upper()} disponíveis')
    for i in range(len(arquivos_extensao)):
        print(f'\t{i+1}\t{arquivos_extensao[i]}')
        opcoes.update({str(i+1):arquivos_extensao[i]})
    return opcoes
        

# Função que retorna uma lista de valores sem repetição

In [36]:
def lista_de_valores_sem_repeticoes(lista_de_dicionarios, chave):
  """
  Recebe uma lista de dicionários.
  A função retorna lista dos valores encontrados para a chave, sem repetições de valores.

  Args:
    lista_de_dicionarios: Uma lista de dicionários.
    chave: A chave a ser pesquisada.

  Returns:
    Uma lista.
  """
  lista_de_valores = set()
  for dicionario in lista_de_dicionarios:
    if chave in dicionario:
      lista_de_valores.add(dicionario[chave])
  return list(lista_de_valores)

# Verifica se a variável é um número

In [37]:
def validacao_numero(pergunta:str, var_tipo: type):
    """Valida e converte variáveis numéricas

    Args:
        pergunta (str): Pergunta realizada pelo usuário para coletar o valor
        var_tipo (type): tipo de variável numérica do valor que será informado pelo usuário

    Returns:
        valor validado e convertido de acordo com a variável var_tipo
    """
    valido = False
    print(pergunta)
    if var_tipo == int:
        while valido != True:
            numero = input(pergunta)
            try:
                int(numero)
                valido = True
            except ValueError:
                print("Informe um valor válido!")
                numero = input(pergunta)
                valido = False
            else:
                return int(numero)
    else:
        while valido != True:
            numero = input(pergunta)
            try:
                float(numero)
                valido = True
            except ValueError:
                print("Informe um valor válido!")
                numero = input(pergunta)
                valido = False
            else:
                return float(numero)



# Funções de Importação

## Função para a leitura de dados de um csv ou json e transformação em uma lista de dicionários

In [38]:
import json
import csv
def importar_dados(extensao:str):
    """ realiza a importação dos dados de arquivos .json ou .csv

    Args:
        extensao (str): extensão do arquivo

    Returns:
        dados(list): lista de dicionários com as variáveis selecionadas para o estudo de caso
    """
    dados = list()
    
    opcoes = arquivos_disponiveis(extensao)
    respostas_validas = list(opcoes.keys())
    pergunta = 'Informe o numero do arquivo que deseja importar'
    str_arquivo = opcoes[validacao_respostas(pergunta, respostas_validas)]
    
    if extensao == '.json':
        with open(str_arquivo, "r", encoding = "utf-8") as f:
            dados = json.load(f) # os arquivos importados via .json já estarão filtrados pelas categorias filtradas pela importação via .csv

    else:
        with open(str_arquivo, "r") as arquivo:
            arquivo_csv = csv.reader(arquivo, delimiter=",")
            next(arquivo_csv)  # pula a primeira linha (cabeçalho)
            for linha in enumerate(arquivo_csv):
                Customer_ID, Age, Gender, Item_Purchased,Category, Purchase_Amount, Location, Size, Color, Season, Review_Rating, Subscription_Status, Payment_Method, Shipping_Type, Discount_Applied, Promo_Code_Used, Previous_Purchases, Preferred_Payment_Method, Frequency_of_Purchases = linha[1]
                #print('ID', Customer_ID,'Idade', Age,'Genero', Gender,'Item',Item_Purchased,
                #                          'Categoria', Category,'Metodo_Pagamento', Payment_Method)
                # 
                dados.append({      'ID': int(Customer_ID), 
                                    'Idade': int(Age), #numérica
                                    'Genero': Gender, #categórica
                                    'Item':Item_Purchased, #categórica
                                    'Categoria': Category, #categórica
                                    'Preco': float(Purchase_Amount), #numérica
                                    'Metodo_Pagamento': Payment_Method, #categórica
                                    'Qtd_Compras': int(Previous_Purchases)#, #numérica
                                    #'Frequencia_Compra': Frequency_of_Purchases #categórica
                                    })
    return dados


## Função para traduzir palavras
É necessário instalar o pacote ``translate``

In [39]:
# (terminal) pip install translate
from translate import Translator
def traduz_palavras(lista_de_palavras):
  """
  Recebe uma lista de palavras em inglês e retorna um dicionário onde 
  a chave é a palavra da lista e o valor é a tradução em português.

  Args:
    lista_de_palavras: Uma lista de palavras em inglês.

  Returns:
    Um dicionário.
  """
  dicionario_de_traducao = {}
  traducao = Translator(to_lang='pt-br', from_lang='english')
  for palavra in lista_de_palavras:
    dicionario_de_traducao[palavra] = traducao.translate(palavra)
  return dicionario_de_traducao


## Traduzindo os valores retirados do CSV

In [40]:
def traduzindo_lista_dict(lista: list):
    """Traduz os valores de uma lista de dicionários

    Args:
        lista (list): lista de dicionários

    Returns:
        lista (list): lista de dicionários traduzida
    """
    for chave in list(lista[0].keys()):
        if type(lista[0][chave]) == str:
            categorias = lista_de_valores_sem_repeticoes(lista, chave)
            dict_traducao = traduz_palavras(categorias)
            for i in range(len(lista)):

                lista[i][chave] = dict_traducao[lista[i][chave]]
    return lista

## Detectar idioma

In [41]:
from langdetect import detect 

def detectar_idioma(dados:list):
    """ A função detecta o idioma dos dados importados e
    questiona ao usuário que ser traduzí-los.
    Se a resposta for positiva, a função traduzindo_lista_dict() será chamada.

    Args:
        dados (list): dados dispostos em listas de dicionários

    Returns:
        dados (list): dados dispostos em listas de dicionários traduzidos para português
    """
    if detect(dados[0]['Metodo_Pagamento']) == 'en': # um exemplo de dados categóricos para verificar o idioma
        pergunta = 'Os dados estão em inglês, deseja traduzir?[S] Sim  [N] Não\t'
        resp = validacao_respostas(pergunta, ['S','N'])
        if resp == 'S':
            print("Essa operação pode demorar, aguarde.")
            return traduzindo_lista_dict(dados)
        return dados



## Menu de importação dos dados

In [42]:
def menu_importar_dados():
    """
        A função coleta do usuário as informações necessárias para realizar a importação de dados.
        Informações coletadas:
        - a extensão do dado que deseja.
            - csv
            - json
            - não importar nenhum arquivo
        - o nome do arquivo

    Return:
        dados (list): dados dispostos em listas de dicionários  
    
    """
    pergunta = """Que tipo de arquivo deseja importar:
      1 \t CSV
      2 \t JSON
      3 \t Não importar nada   """
    resp =  validacao_respostas(pergunta, ['1','2','3'])
    if   resp == '1':
        dados = importar_dados('.csv')
        detectar_idioma(dados)
    elif resp == '2':
        dados = importar_dados('.json')
        detectar_idioma(dados)
    else:
        dados = list()
    return dados

# Função para exportar uma lista de dicionários para um arquivo .json

In [58]:
import json
import os.path
def exportar_json(lista_vendas: list):
    """
    A função exporta a lista_vendas para um arquivo.
    O arquivo será nomeado pelo usuário
        Caso o arquivo já exista na pasta, o usuário será questionado se quer sobrescrever.
    O arquivo será no formato .json
    Será importado na mesma pasta que se encontra o script
    Args:
        lista_vendas (list): lista de dicionários com as informações de venda
    """
    nome_arquivo = input("Informe o nome do arquivo a ser criado:\t").lower()
    

    # retirando os possíveis espaços do nome
    qtd_espaco = nome_arquivo.count(' ')
    if qtd_espaco > 0:
        nome_arquivo =  nome_arquivo.replace(" ", "_", qtd_espaco)
    nome_arquivo += '.json'
    
    if os.path.exists(nome_arquivo) == True:
        pergunta = 'Já existe um arquivo com esse nome.\nDeseja sobrescrever?[S]  Sim\t [N] Não\t'
        resp_sobrescrever = validacao_respostas(pergunta, ['S','N'])
        if resp_sobrescrever == 'N':
            print('O arquivo não foi modificado.')
            resp_sobrescrever = validacao_respostas(pergunta, ['S','N'])
            return
    try:
        json_vendas = json.dumps(lista_vendas, indent=2,ensure_ascii=False)
        arquivo = open(nome_arquivo, 'w', encoding = "utf-8") 
        arquivo.write(json_vendas) 
        arquivo.close()
        print("Arquivo exportado com sucesso!")
    except:
        print("Erro ao exportar arquivo")
    return

# Listagem de dados

## Lista simples de todos os dados da lista

In [43]:
def listar_dados(dados: list):
    """Imprime na tela uma lista simples com um registro por linha

    Args:
        dados (list): lista de dicionários
    """
    print("\t".join(list(dados[0].keys())))
    for j in range(len(dados)):
        lista_linha = [str(list(dados[j].values())[i]) for i in range(len(list(dados[j].values())))]
        print("\t".join(lista_linha))


## Função que exibe os registros

In [44]:
def exibir_dados(dados: list, ids: list ):
    """Exibe os dados de maneira detalhada, cada chave em uma linha

    Args:
        dados (list): lista de dicionários
        ids (list): identificadores dos registros
    """
    lista_indices = [i for i in range(len(dados)) if dados[i]['ID'] in ids]
    print("Listagem:")
    lista_chaves = list(dados[0].keys())
    for i in lista_indices:
        print(' - - - - - - - - - - - - - - - - - - - - - -')
        lista_valores = list(dados[i].values())
        for j in range(len(lista_chaves)):
            print(f'{lista_chaves[j].upper()}: {lista_valores[j]}')

## Menu de listagem de registro

In [45]:
def menu_lista(dados: list):
    """Solicita do usuários informações para a listagem dos dados

    Args:
        dados (list): lista de dicionários
    """
    pergunta ='''Opções de Listagem:
    [1] Lista de todos os registros
    [2] Lista de registros específicos - via ID
    [3] Lista de registros com ids sequenciais'''
    resp = validacao_respostas(pergunta, ['1','2','3'])
    if resp == '1':
        listar_dados(dados)
    elif resp == '2':
        pergunta = "Informe o ID que seja visualizar registro e 0 para sair"
        numero = 1# para inicializar o while
        lista_ids = list()
        while True:
            numero = validacao_numero(pergunta, int)
            if numero == 0:
                break
            lista_ids.append(numero)
        if len(lista_ids) == 0:
            print('Não há registros para exibir!')
        else:
            exibir_dados(dados,lista_ids)
    else:
        pergunta = "Informe o ID inicial da sequência:"
        numero_inicial = validacao_numero(pergunta, int)
        while numero_inicial <=0:
            print('Informe valores positivos de ID')
            numero_inicial = validacao_numero(pergunta, int)
            
        pergunta = "Informe o ID final da sequência:"
        numero_final = validacao_numero(pergunta, int)
        while numero_final <= numero_inicial:
            print('Informe um valor maior que o ID inicial')
            numero_final = validacao_numero(pergunta, int)
        exibir_dados(dados,list(range(numero_inicial, numero_final)))
        

# Funções de Editar registros

## edição do registro

In [46]:
def editar_registros(dados: list, chave: str, valor, id:int):
    """A função recebe as informações para editar o registro e retorna os dados atualizados

    Args:
        dados (list): lista de dados
        chave (str): chave das categorias do registro
        valor (_type_): valor do registro
        id (int): identificador do registro

    Returns:
        dados (list): lista de dicionários atualizado
    """
    indice =  [i for i in range(len(dados)) if dados[i]['ID'] == id][0]
    dados[indice][chave] = valor
    return dados

## seleção da chaves que se quer editar

In [47]:
def selecao_chave_edicao(dados: list, id : int):
    """Solicita do usuário a informação dos campos que deseja editar

    Args:
        dados (list): lista de dicionários
        id (int): identificador do registro

    """
    print('Qual ou quais campos deseja editar, separando-os por vírgula')
    indice =  [i for i in range(len(dados)) if dados[i]['ID'] == id][0]
    chaves = list(dados[indice].keys())
    lista_opcoes = [1]
    i = 1
    for chave in chaves[1:]:
        print(f'{i}\t{chave}')
        i +=1
        lista_opcoes.append(i)
    print(f'{i}\tNovo campo')
    chaves
    lista_selecionados     = (input().replace(' ','')).split(',')
    
    lista_opcoes_str       = list(map(str, lista_opcoes))
    
    errados = [i for i in lista_selecionados if not i in lista_opcoes_str]
    if len(errados) >= 1:
        if len(errados) == 1:
            print(f'O campo "{" ".join(errados)}" foi desconsiderado por ser inválido.')
        else:
            print(f'Os campos\n{" - ".join(errados)}\nforam desconsiderados por serem inválidos.')

        pergunta = 'Deseja refazer a seleção dos campos?\n[S] Sim\n[N] Não'
        resp = validacao_respostas(pergunta, ['S','N'])
    
        if resp == 'S':
            selecao_chave_edicao(dados, id)
    
    lista_selecionados_corrigidos = [i for i in lista_selecionados if not i in errados  ]
    lista_selecionados_corrigido_int = list(map(int, lista_selecionados_corrigidos ))
    if max(lista_selecionados_corrigido_int) == len(chaves):
        novo_campo = input("Foi selecionado a opção 'Novo Campo'\n Nomeei novo campo:")
        chaves.append(novo_campo)
    
    print('Campos selecionados para a edição:')
    for i in lista_selecionados_corrigido_int:
        chave = chaves[i]
        pergunta = f'Informe o novo valor para o campo {chave.upper()}'
        if dados[indice][chave] == None:
            pergunta = f'''Que tipo de informação será armazenada no campo {chave}?
            [1] Categórico
            [2] Números Inteiros
            [3] Números Decimais'''
            resp = validacao_respostas(pergunta, ['1','2','3'])
            if   resp == '2':
                valor = validacao_numero(pergunta, int)
            elif resp == '3':
                valor = validacao_numero(pergunta, float)
            else:
                valor = input(pergunta).title()
        elif type(dados[indice][chave]) == int:
            valor = validacao_numero(pergunta, int)
        elif type(dados[indice][chave]) == float:
            valor = validacao_numero(pergunta, float)
        else:
            valor = input(pergunta).title()
        return editar_registros(dados, chave, valor, id)
    print('O registro foi atualizado!')
    exibir_dados(dados, [id])
    return dados


## seleção do registro que se quer alterar

In [48]:
def selecao_id_edicao(dados: list):
    """Solicita do usuário o identificador do registro que deseja alterar

    Args:
        dados (list): lista de dicionários

    """
    print('Para fazer a edição de um registro, é necessário informar o seu ID')
    pergunta = '''Deseja listar os registros?
    [S]\t Sim
    [N]\t Não
    '''
    resp = validacao_respostas(pergunta, ['S','N'])
    if resp == 'S':
        listar_dados(dados)
    pergunta = 'Informe o ID do registro que deseja editar'
    resp = validacao_numero(pergunta, int)
    return selecao_chave_edicao(dados, resp)

# Função Remover

In [49]:
import json
def remover_arquivo(dados):
    """"
    Remove dicionários de um arquivo json através do ID inserido.

    Parâmetros:
    dados(list): Uma lista de dicionários.
    """
    
    #with open('dados_vendas_traduzido.json', 'r') as arquivo:
    #    dados = json.load(arquivo)
        
    entrada_valida = False
    while not entrada_valida:
        id_input = input('Digite o ID que deseja remover: ')

        if id_input.isdigit():
            id_remover = int(id_input)
            entrada_valida = True

        else:
            print("Por favor, insira apenas números.")
    

    dados2 = [item for item in dados if item['ID'] != id_remover]

    if len(dados2) < len(dados):
        print('Dados removidos com sucesso!')
    else:
        print(f'O ID {id_remover} não foi encontrado!')

    #with open('dados_vendas_traduzido.json', 'w') as arquivo:
    #    json.dump(dados2, arquivo, indent=2)
    return dados2

# Funções para filtro

## Função de filtro

In [50]:
def filtro(lista: list, categoria:str, valor_categoria, tipo:str):
    """Função que realiza os filtros de acordo com os parâmetros de entrada passados

    Args:
        lista (list): dados dispostos em listas de dicionários
        categoria (str): chave da categoria que se deseja filtrar 
        valor_categoria: valor utilizado com o operador comparativo
        tipo (str): nome do operador comparativo

    Returns:
        filtro (list): dados filtrados dispostos em listas de dicionários
    """

    if   tipo == 'Maior que':
        filtro = list(filter(lambda x: x[categoria] >  valor_categoria, lista))
    elif tipo == 'Maior ou igual':
        filtro = list(filter(lambda x: x[categoria] >= valor_categoria, lista))
    elif tipo == 'Menor que':
        filtro = list(filter(lambda x: x[categoria] <  valor_categoria, lista))
    elif tipo == 'Menor ou igual':
        filtro = list(filter(lambda x: x[categoria] <= valor_categoria, lista))
    elif tipo == 'Igual a':
        filtro = list(filter(lambda x: x[categoria] == valor_categoria, lista))
    else:       # Diferente de
        filtro = list(filter(lambda x: x[categoria] != valor_categoria, lista))
    return filtro


### Filtro Categórico

In [51]:
def filtro_categorico(dados:list, chave:str):
    """A função coleta do usuário qual operação se deseja realizar no filtro de variáveis categóricas

    Args:
        dados (list): dados dispostos em listas de dicionários
        chave (str): chave da categoria que se deseja filtrar
        As opções de filtros são
                - Diferente de
                - Igual a
    Returns:
        dados   (list): dados dispostos em listas de dicionários
        chave    (str): chave da categoria que se deseja filtrar  
        valor    (str): valor para realizar a operação de comparação 
        operacao (str): Nome da operação
    """

    pergunta = f'Informe o número da opção você quer filtrar dentro de {chave}: '
    opcoes = lista_de_valores_sem_repeticoes(dados, chave)

    respostas_validas = list(range(1,len(opcoes)+1))
    respostas_validas = list(map(str, respostas_validas))
    print(f'\nLista de opções dentro da categoria {chave}')
    for i in range(len(opcoes)):
        print(1+i,'\t',opcoes[i])

    resp = validacao_respostas(pergunta, respostas_validas)
    valor = opcoes[int(resp)-1]
    pergunta = f'''Como deseja filtrar os dados dentro de {chave}
        [1] Para valores diferentes de {valor}
        [2] Para valores iguais a de {valor}.'''
    resp = validacao_respostas(pergunta, ['1','2'])

    if resp == '1':
        operacao = 'Diferente de'
    else:
        operacao = 'Igual a'
    return dados, chave, valor, operacao

### Filtro Numérico

In [52]:
def filtro_numerico(dados:list, chave:str, var_tipo: type):
    """A função coleta do usuário qual operação se deseja realizar no filtro de variáveis numéricas

    Args:
        dados (list): dados dispostos em listas de dicionários
        chave (str): chave da categoria que se deseja filtrar
        var_tipo(type): tipo da variável (int ou float) que será utilizada para a comparação
        As opções de filtros são
                - Diferente de
                - Igual a
                - Maior que
                - Maior ou Igual
                - Menor que
                - Menor ou Igual 
    Returns:
        dados   (list): dados dispostos em listas de dicionários
        chave    (str): chave da categoria que se deseja filtrar  
        valor  (float): valor para realizar a operação de comparação 
        operacao (str): Nome da operação
    """
    operadores_comparacao = ['Maior que','Maior ou igual',
                            'Menor que','Menor ou igual',
                            'Igual a','Diferente de']
    respostas_validas = list(range(1,len(operadores_comparacao)+1))
    respostas_validas = list(map(str, respostas_validas))

    for i in range(len(respostas_validas)):
        print(1+i,'\t',operadores_comparacao[i])
    pergunta = f'Informe o número da opção do operador para usar na categoria {chave}: '
    resp = validacao_respostas(pergunta, respostas_validas)
    operacao = operadores_comparacao[int(resp)-1]
        
    pergunta = 'Informe um valor para realizar a comparação: '
    valor = validacao_numero(pergunta, var_tipo)
    print(chave, valor, operacao)
    return dados, chave, valor, operacao

## Menu filtros

In [53]:
def opcoes_filtros(dados:list):
    """A função coleta do usuário as informações necessárias para realizar a filtragem de dados.
        Informações coletadas:
        A categoria (chave) que se quer usar para realizar o filtro.
        A operação que se deseja realizar será coletada a partir de duas funções
        - filtro_categorico para variáveis categóricas
        - filtro_numerico para variáveis numéricas (float e int)
        E o valor necessário para tal.
    Args:
        dados (list): dados dispostos em listas de dicionários
    """
    tamanho = len(list(dados[0].keys())[1:])
    lista_categorias = list(dados[0].keys())[1:]
    respostas_validas = list(range(1,tamanho+1))
    respostas_validas = list(map(str, respostas_validas))

    print('\nCategorias Disponíveis para consulta:')
    for i in range(tamanho):
        print(1+i,'\t',lista_categorias[i])
    pergunta = 'Informe o número da categoria que deseja filtrar:'
    resp = validacao_respostas(pergunta, respostas_validas)
    chave = str(lista_categorias[int(resp)-1])

    if type(dados[0][chave]) == str:
        dados, chave, valor, operacao = filtro_categorico(dados, chave)
          
    elif type(dados[0][chave]) == int: 
        dados, chave, valor, operacao = filtro_numerico(dados, chave, int)
    else: # float
        dados, chave, valor, operacao = filtro_numerico(dados, chave, int)
        
    print(f'''Detalhes do filtro:
              Categoria: {chave}
              Valor:     {valor}
              Operação:  {operacao}
              ''')
    dados_filtrados = filtro(dados, chave, valor, operacao)
    
    pergunta = '''O que deseja fazer com os dados filtros:
    [1] Exibir na tela
    [2] Exportar
    '''
    resp = validacao_respostas(pergunta, ['1','2'])
    if resp == '1':
        listar_dados(dados_filtrados)
    else:
        exportar_json(dados_filtrados)

## Função adicionar registro
-> utilizar a função: ``editar_registros(dados, chave, valor, id)``

dados: lista de dicionários

chave: a chave do dicionário

valor: valor da chave

id: identificador - não é o mesmo que o índice de lista

-> a função `validacao_numero(pergunta:str, var_tipo: type)` para a validação de numéricos
``pergunta`` pode ser algo como "informe o valor" e o ``var_tipo`` é float, 
a função vai verificar se é ``float``. Se não for, vai repetir a ``pergunta`` até a resposta seja um float, que é retornado.

- encontrar o maior número de id e insere o registro no próximo valor
- chamar a função `editar_registros` para cada chave e valor
- levar em conta se a lista de dicionários está vazia


In [54]:
def adicionar_dados_manualmente(dados:list):
    """Adiciona todos os campos iniciais de um registro e o adiciona na lista de registros

    Args:
        dados (list): lista de dicionários

    Returns:
        dados (list): lista de dicionários atualizados
    """
    if len(dados) == 0:
        novo_id = 1
    else:
        novo_id = dados[-1]['ID'] + 1

    #metodos_pagamento = list(set(item['Metodo_Pagamento'] for item in dados))
    #categorias = list(set(item['Categoria'] for item in dados))

    idade = validacao_numero("Idade: ", int)
    genero = validacao_respostas("Gênero ([H] Homem/[M] Mulher): ", ['H', 'M'])
    if genero == 'H':
        genero = 'Homem'
    else:
        genero = 'Mulher'
    item = input("Item: ")

    categoria = input('Categoria:')
    preco = validacao_numero("Preço: ", float)
    
    metodo_pagamento = input(f'Método de Pagamento:')
    qtd_compras = validacao_numero("Quantidade de Compras: ", int)

    novo_dado = {
        'ID': novo_id,
        'Idade': idade,
        'Genero': genero,
        'Item': item.title(),
        'Categoria': categoria.title(),
        'Preco': preco,
        'Metodo_Pagamento': metodo_pagamento.title(),
        'Qtd_Compras': qtd_compras
    }

    
    dados.append(novo_dado)
    print("Novo registro adicionado com sucesso!")
    return dados

# Estatísticas

## Função para calcular ticket médio

In [55]:
def calcular_ticket_medio(dados):
    """
    O ticket médio é um indicador usado para avaliar o gasto médio dos clientes.
    Para calculá-lo, deve-se dividir o faturamento total das vendas pelo número de vendas do intervalo de tempo em questão.
    """
    faturamento = 0
    for item in dados:
        faturamento = sum([faturamento, item["Preco"]])
    
    transacoes = len(dados)

    if transacoes == 0:
        # Para evitar divisões por zero
        return 0
    else:
        ticket_medio = faturamento / transacoes
        return ticket_medio

## Regra de Negócio

In [56]:
def grupo_clientes(dados):
    """
    A clusterização de clientes permite agrupá-los por alguma semelhança.
    É um meio para entender o perfil de consumo dos clientes para que, de forma estratégica,
      sejam criados ofertas personalizadas.
    Nessa função os clientes serão separados em dois grupos:
    - A: perfil de consumo a partir do ticket médio
    - B: consome abaixo do ticket médio
    """
    for itens in dados:
        if Preco >= calcular_ticket_medio(dados):
            adicionar_registro("Grupo" : "A")
        else:
            adicionar_registro("Grupo" : "B")

SyntaxError: invalid syntax (2204531749.py, line 12)

In [ ]:

def add_grupo_cliente(dados: list, valor, categorias:list, chave:str):
    #adicionando nova chave e valores utilizando valores para separar as categorias
    for item in range(len(dados)):
        if dados[item]['Preco'] >= valor:
            dados = editar_registros(dados, chave, categorias[0], dados[item]['ID'])
        else:
            dados = editar_registros(dados, chave, categorias[1], dados[item]['ID'])
    return dados

## para testar a funcao
#dados = importar_dados('.json')
#add_grupo_cliente(dados, 231.60, ['A','B'], 'Grupo')




# Função para exportar uma lista de dicionários para um arquivo .json

In [ ]:
import json
import os.path
def exportar_json(lista_vendas: list):
    """
    A função exporta a lista_vendas para um arquivo.
    O arquivo será nomeado pelo usuário
        Caso o arquivo já exista na pasta, o usuário será questionado se quer sobrescrever.
    O arquivo será no formato .json
    Será importado na mesma pasta que se encontra o script
    Args:
        lista_vendas (list): lista de dicionários com as informações de venda
    """
    nome_arquivo = input("Informe o nome do arquivo a ser criado:\t").lower()
    

    # retirando os possíveis espaços do nome
    qtd_espaco = nome_arquivo.count(' ')
    if qtd_espaco > 0:
        nome_arquivo =  nome_arquivo.replace(" ", "_", qtd_espaco)
    nome_arquivo += '.json'
    
    if os.path.exists(nome_arquivo) == True:
        pergunta = 'Já existe um arquivo com esse nome.\nDeseja sobrescrever?[S]  Sim\t [N] Não\t'
        resp_sobrescrever = validacao_respostas(pergunta, ['S','N'])
        if resp_sobrescrever == 'N':
            print('O arquivo não foi modificado.')
            resp_sobrescrever = validacao_respostas(pergunta, ['S','N'])
            return
    try:
        json_vendas = json.dumps(lista_vendas, indent=2,ensure_ascii=False)
        arquivo = open(nome_arquivo, 'w', encoding = "utf-8") 
        arquivo.write(json_vendas) 
        arquivo.close()
        print("Arquivo exportado com sucesso!")
    except:
        print("Erro ao exportar arquivo")
    return

# Função Principal

In [57]:
def main():
    
    # importando dados iniciais
    dados = menu_importar_dados()
    # iniciando o loop do menu
    
    continua = 'S'
    while continua =='S':
        print('\n - - - - - - - - - - - - - - - - - \n')

        pergunta = """Qual ação deseja realizar?
Para adicionar um registro             --- A
Para remover um registro               --- R
Para editar um registro                --- E
Para leitura de registros              --- L
Para mostrar a regra de negócio        --- N
Para realizar estatística              --- S
Para realizar filtro                   --- F
Para exportar dados para JSON          --- J
Para trocar banco de dados             --- T
Para sair do menu                      --- Q
        """
        opcao = validacao_respostas(pergunta, ['A','R','E','L','N','S','F','J','T','Q'])
        if   opcao == 'A': 
            dados = adicionar_dados_manualmente(dados)
        elif opcao == 'R': 
            dados = remover_arquivo(dados)
        elif opcao == 'E':
            dados = selecao_id_edicao(dados)
        elif opcao == 'L':
            menu_lista(dados)
        elif opcao == 'N':
            ticket_medio = calcular_ticket_medio(dados)
            print(f'Valor do Ticket Médio:\nR$ {ticket_medio:.2f}\n')
            #print('''
            #      Como regra de negócio, iremos classificar os clientes em duas categorias:
            #      Grupo A -> Clientes com consumo igual ou maior que o Ticket Médio
            #      Grupo B -> Clientes com consumo inferior ao do Ticket Médio
            #      
            #      A categorização foi incluida nos registros.
            #      ''')
            #add_grupo_cliente(dados, ticket_medio, ['A','B'], 'Grupo')    
        elif opcao == 'S': 
            pass
        elif opcao == 'F':
            opcoes_filtros(dados)
        elif opcao == 'J':
            exportar_json(dados)
        elif opcao == 'T':
            dados = menu_importar_dados()
        else:
            continua = 'N'
    print('Sistema Encerrado!')   
main()
    

Que tipo de arquivo deseja importar:
      1 	 CSV
      2 	 JSON
      3 	 Não importar nada   

 - - - - - - - - - - - - - - - - - 

Qual ação deseja realizar?
Para adicionar um registro             --- A
Para remover um registro               --- R
Para editar um registro                --- E
Para leitura de registros              --- L
Para mostrar a regra de negócio        --- N
Para realizar estatística              --- S
Para realizar filtro                   --- F
Para exportar dados para JSON          --- J
Para trocar banco de dados             --- T
Para sair do menu                      --- Q
        
Idade: 
Gênero ([H] Homem/[M] Mulher): 
Preço: 
Quantidade de Compras: 
Novos dados adicionados com sucesso!

 - - - - - - - - - - - - - - - - - 

Qual ação deseja realizar?
Para adicionar um registro             --- A
Para remover um registro               --- R
Para editar um registro                --- E
Para leitura de registros              --- L
Para mostrar a regra de n